In [2]:
# Define rooms

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
    "name": "outside",
    "type": "room",
}

outside = {
  "name": "outside"
}
# Define doors

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}

# define items in Game Room
guitar = {
    "name": "guitar",
    "type": "furniture",
}

couch = {
    "name": "couch",
    "type": "furniture",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}
piano = {
    "name": "piano",
    "type": "furniture",
}

# Define items in Bedroom 1

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

sad_clown = {
    "name": "sad clown painting",
    "type": "furniture",
}

key_b_bedroom_1 = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

# Define items in Bedroom 2
double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

key_c_bedroom_2 = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d_bedroom_2 = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

# Define items in Living Room
dining_table = {
    "name": "dining table",
    "type": "furniture",
}

all_rooms = [game_room, outside, bedroom_1, bedroom_2, living_room]
all_doors = [door_a, door_b, door_c, door_d]
# define which items/rooms are related
object_relations = {
    "game room": [couch, piano, door_a, guitar],
    "piano": [key_a],
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "bedroom 1": [door_a, door_b, door_c, queen_bed, sad_clown],
    "queen bed": [key_b_bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "bedroom 2": [double_bed, dresser, door_b],
    "double bed": [key_c_bedroom_2],
    "dresser": [key_d_bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside],
    "living room": [dining_table, door_d, door_c]
}
# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.
INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("\033[1m\033[33mYou wake up on a couch and find yourself in a strange house with no windows which you have never been to before.\nYou don't remember why you are here and what had happened before.\nYou feel some unknown danger is approaching and you must get out of the house, NOW!\033[0m")
    linebreak()
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("\033[92mCongrats! You escaped the room!\033[0m")
    else:
        # capitalized the room name so it's easier for the user to read
        print("\nYou are now in " + str(room["name"]).upper())
        # split this text over two lines using \n
        intended_action = input("\nWhat would you like to do?\nType 'explore' or 'examine'? ").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("\nWhat would you like to examine?\n").strip().lower())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'. ")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    # capitalized the room name so it's easier for the user to read
    # added \n before the list of items in the room are printed
    # slightly modified the text
    #print("You explore the room. This is " + str(room["name"]).upper() + ". You find the following items\n" + "\n".join(items))
    print("\nYou explore the room. This is " + str(room["name"]).upper() + ". You find the following items:\n\n" + "\n".join([item.upper() for item in items]))


def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if item_name == "guitar":  # Special case for the guitar
                    output += "\033[91m\nYou hear a faint melody coming from the guitar, as you draw closer to it, but something tells you to better not touch it.\033[0m"
                elif item_name == "sad clown painting": # Special case for painting
                    output += "\033[91m\nYou observe the brittle oil paint and suddenly you become very, very sad. Better look for something else, before you burst into tears...😢\033[0m"
                elif item_name == "couch": # Special case for couch
                    output += "\033[91m\nThe couch where you woke up smells musty and you can still feel the stiffness in your bones. You must have been there for ages. You move on.\033[0m"
                elif(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "\033[95mYou find " + item_found["name"] + ".\033[0m"
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break
    if(output is None):
        print("The item you requested is not found in the current room.")
    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):
        linebreak()
        play_room(next_room)

    else:
        play_room(current_room)
game_state = INIT_GAME_STATE.copy()
start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before.
You don't remember why you are here and what had happened before.
You feel some unknown danger is approaching and you must get out of the house, NOW!




You are now in GAME ROOM

What would you like to do?
Type 'explore' or 'examine'? explore

You explore the room. This is GAME ROOM. You find the following items:

COUCH
PIANO
DOOR A
GUITAR

You are now in GAME ROOM

What would you like to do?
Type 'explore' or 'examine'? examine

What would you like to examine?
piano
You examine piano. You find key for door a.

You are now in GAME ROOM

What would you like to do?
Type 'explore' or 'examine'? examine

What would you like to examine?
door a
You examine door a. You unlock it with a key you have.
Do you want to go to the next room? Enter 'yes' or 'no'explore

You are now in GAME ROOM

What would you like to do?
Type 'explore' or 'examine'? explore

You explore the room. This is